In [1]:
from __future__ import print_function
import json
from kafka import KafkaProducer, KafkaClient
import tweepy
from time import sleep

In [2]:
# Twitter Credentials Obtained from http://dev.twitter.com
consumer_key = 'YOUR_TOKEN'
consumer_secret = 'YOUR_TOKEN'
access_key = 'YOUR_TOKEN'
access_secret = 'YOUR_TOKEN'

# Words to track
WORDS = ['COVID']

topic_name = 'covid'

class StreamListener(tweepy.StreamListener):
    # This is a class provided by tweepy to access the Twitter Streaming API.

    def on_connect(self):
        # Called initially to connect to the Streaming API
        print("You are now connected to the streaming API.")

    def on_error(self, status_code):
        # On error - if an error occurs, display the error / status code
        print("Error received in kafka producer " + repr(status_code))
        return True # Don't kill the stream

    def on_data(self, data):
        """ This method is called whenever new data arrives from live stream.
        We asynchronously push this data to kafka queue"""
        try:
            producer.send(topic_name, data.encode('utf-8'))
            print(data)   
                  
            sleep(3)
            
        except Exception as e:
            print(e)
            return False
        return True # Don't kill the stream

    def on_timeout(self):
        return True # Don't kill the stream

# Kafka Configuration
producer = KafkaProducer(bootstrap_servers=['localhost:9092'])

# Create Auth object
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

# Set up the listener. The 'wait_on_rate_limit=True' is needed to help with Twitter API rate limiting.
listener = StreamListener(api=tweepy.API(wait_on_rate_limit=True, wait_on_rate_limit_notify=True, timeout=20, retry_delay=5, retry_count=10, retry_errors=set([401, 404, 500, 503])))
stream = tweepy.Stream(auth=auth, listener=listener)
print("Tracking: " + str(WORDS))
stream.filter(track=WORDS, languages = ['en'])

Tracking: ['COVID']
You are now connected to the streaming API.
{"created_at":"Sun May 10 13:41:58 +0000 2020","id":1259478791333838848,"id_str":"1259478791333838848","text":"RT @PeterKGeoghegan: \"Stay alert\" doesn't mean \"stay at home\" it means \"stay alert\".\n\nBut how do I \"stay alert\" when I go out when viruses\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":923942883304067072,"id_str":"923942883304067072","name":"Ian Spindley #stopclimatechange","screen_name":"IanSpindley","location":"Criccieth, Wales","url":null,"description":"Dad\/husband\/son. Work marketing\/communications\/media manager. Love birds\/wildlife\/countryside\/environment sustainability. And stories\/music\/politics with beer.","translator_type":

KeyboardInterrupt: 